In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,precision_score,confusion_matrix
from sklearn.metrics import precision_recall_curve ,brier_score_loss, accuracy_score, f1_score, recall_score
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import SMOTENC

from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
import scipy
import optuna

import warnings
warnings.filterwarnings('ignore')

In [2]:
# path to files
x_train_path =  "/kaggle/input/ogd-goi/X_Train_Data_Input.csv"
y_train_path = "/kaggle/input/ogd-goi/Y_Train_Data_Target.csv"
x_test_path = "/kaggle/input/ogd-goi-test/X_Test_Data_Input.csv"
y_test_path = "/kaggle/input/ogd-goi-test/Y_Test_Data_Target.csv"

# Data Preprocessing

## handling null values

In [3]:
def file_read_and_col_verification(path_x: str):
        print("IN FILE READ FUNCTION")
        xcolumns = ["ID", "Column0", "Column1", "Column2", "Column3", "Column4", "Column5", "Column6", "Column7", 
                "Column8", "Column9", "Column10", "Column11", "Column12", "Column13", "Column14", "Column15", 
                "Column16", "Column17", "Column18", "Column19", "Column20", "Column21"]
        try:
            xdf = pd.read_csv(path_x, header=0)
        except Exception as e:
            print(e)
            return 'file read error'
    
        cols = list(xdf.columns)
        if cols != xcolumns:
            return "columns mismatch, check if this is the correct file"
        xdf.drop(['Column9','Column14'], inplace=True, axis=1)  # almost 50% null
        xdf.drop(['Column4', 'Column11', 'Column12', 'Column13'], axis=1, inplace=True)  # highly correlated
        xdf
        return xdf


def df_x_cleaner(path: str, cols_list: list) -> pd.DataFrame:
    
    xdf = file_read_and_col_verification(path)
    ID = xdf['ID']
    xdf.drop(['ID'], axis=1, inplace=True)
    xdf['Column0'] = xdf['Column0'].fillna(1)
    imputed_df_sample = xdf.sample(n=100000, random_state=42)
    conditions = [imputed_df_sample[col].notnull() for col in cols_list]
    combined_condition = np.logical_and.reduce(conditions)
    df_not_missing = imputed_df_sample[combined_condition]
    for i in range(len(cols_list)):
        try:
            X_train_sub = df_not_missing.drop(cols_list, axis=1)
            y_train_sub = df_not_missing[cols_list[i]]
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train_sub, y_train_sub.values.ravel())
            X_full_missing = xdf[xdf[cols_list[i]].isnull()].drop(cols_list, axis=1)
            predicted_values = model.predict(X_full_missing)
        except Exception as e:
            print(e)
            return None
        xdf.reset_index(inplace=True)
        Temp_col = xdf[xdf[cols_list[i]].isnull()][['index', cols_list[i]]].reset_index(drop=True)
        Temp_col['predicted'] = np.nan  # Placeholder for predicted values
        prd = pd.DataFrame(predicted_values, columns=['predicted'])
        prd.reset_index(drop=True, inplace=True)  # Ensure indices match
        Temp_col['predicted'] = prd['predicted']
        xdf = xdf.set_index('index').combine_first(Temp_col.set_index('index')).reset_index()
        xdf[cols_list[i]] = xdf[cols_list[i]].fillna(xdf['predicted'])
        xdf.drop(['predicted'], axis=1, inplace=True)
        desired_order = ["Column0", "Column1", "Column2", "Column3", "Column5", "Column6", "Column7", 
                             "Column8", "Column10", "Column15","Column16", "Column17", "Column18", "Column19", 
                             "Column20", "Column21"]
        xdf = xdf[desired_order]
    xdf['ID'] = ID
    xdf.drop("ID",axis=1)
    return xdf

In [4]:
x_train_cleaner = df_x_cleaner(x_train_path, ['Column3', "Column5", "Column6", "Column8", "Column15"]).drop("ID",axis=1)
x_test_cleaner = df_x_cleaner(x_test_path, ['Column3', "Column5", "Column6", "Column8", "Column15"]).drop("ID",axis=1)

x_train_cleaner[['Column0','Column2','Column16','Column18']]= x_train_cleaner[['Column0','Column2','Column16','Column18']].astype(int)
x_test_cleaner[['Column0','Column2','Column16','Column18']]= x_test_cleaner[['Column0','Column2','Column16','Column18']].astype(int)

IN FILE READ FUNCTION
IN FILE READ FUNCTION


##  Scaling, clipping, and data normalisation

In [5]:
def process_skew_outlier(data, mode='train', version='initial'):
    def Robust_scal(df):
        scaler = RobustScaler()
        df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
        return df_scaled
    def log_scaling(df, columns):
        for ColumnName in columns:
            df[ColumnName] = np.log1p(df[ColumnName])
        return df
    def clipping_negative_Skew_outlier(df, columns):   # Clipping negative skew outliers for certain columns
        for col in columns:
            lower_cap = df[col].quantile(0.01)  # Lower cap at the 1st percentile
            upper_cap = df[col].quantile(0.99)  # Upper cap at the 99th percentile
            df[col] = df[col].clip(lower=lower_cap, upper=upper_cap)
        return df
    def negative_skew_scaling(df, columns): # Scaling for negatively skewed columns
        for col in columns:
            df[col] = df[col] ** 2
        return df
    if mode == 'train' and version == 'initial': # Define column selections based on mode and version
        log_columns = ["Column0", "Column5", "Column6", "Column7", "Column8", "Column17", "Column18"]
        clip_columns = ["Column0", "Column5", "Column6", "Column7", "Column15", "Column8", "Column17", "Column18"]
        skew_columns = ["Column8", "Column3"]
    elif mode == 'train' and version == 'update':
        log_columns = ["Column5", "Column6", "Column7", "Column8"]
        clip_columns = ["Column5", "Column6", "Column7", "Column15", "Column8"]
        skew_columns = ["Column8", "Column3"]
    elif mode == 'test' and version == 'initial':
        log_columns = ["Column0", "Column5", "Column6", "Column7", "Column8", "Column17", "Column18"]
        skew_columns = ["Column8", "Column3"]
    elif mode == 'test' and version == 'update':
        log_columns = ["Column5", "Column6", "Column7", "Column8"]
        skew_columns = ["Column8", "Column3"]
    else:
        print('wrong parameters')
        return
    if mode == 'train': # Apply transformations
        if version == 'initial':
            rbdf = negative_skew_scaling(
                clipping_negative_Skew_outlier(
                    Robust_scal(
                        log_scaling(data, log_columns)),
                    clip_columns),
                skew_columns)
        else:  # version == 'update'
            rbdf = negative_skew_scaling(
                clipping_negative_Skew_outlier(
                    Robust_scal(
                        log_scaling(data, log_columns)),
                    clip_columns),
                skew_columns)
    else:  # mode == 'test'
        rbdf = negative_skew_scaling(
            Robust_scal(
                log_scaling(data, log_columns)),
            skew_columns)

    return rbdf


In [6]:
x_train_cleaned_scaled_updated = process_skew_outlier(x_train_cleaner, mode='train', version='initial')
x_test_cleaned_scaled_updated = process_skew_outlier(x_test_cleaner,mode='test', version='update')

## SMOTE

In [7]:
def apply_smote_Columns(X_train, y_train):
        print("Smoting the data")
        categorical_columns = ['Column0','Column10','Column16','Column17','Column18','Column19','Column20','Column21']
        smote =  SMOTENC(categorical_features=categorical_columns, random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
        return X_resampled, y_resampled
    

In [8]:
ytrain = pd.read_csv(y_train_path,header=0).drop('ID',axis=1)
ytest = pd.read_csv(y_test_path,header=0).drop('ID',axis=1)


In [9]:
x_smoted_updated_train_data,y_somted_updated_train_data = apply_smote_Columns(x_train_cleaned_scaled_updated, ytrain)

Smoting the data


In [10]:
print(x_smoted_updated_train_data.columns) 
print(x_test_cleaned_scaled_updated.columns)

Index(['Column0', 'Column1', 'Column2', 'Column3', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column10', 'Column15', 'Column16', 'Column17',
       'Column18', 'Column19', 'Column20', 'Column21'],
      dtype='object')
Index(['Column0', 'Column1', 'Column2', 'Column3', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column10', 'Column15', 'Column16', 'Column17',
       'Column18', 'Column19', 'Column20', 'Column21'],
      dtype='object')


In [11]:
# 1. Prepare the data
x_train = x_smoted_updated_train_data
y_train = y_somted_updated_train_data
x_test = x_test_cleaned_scaled_updated

# Model training , testing and validation
## only xgboostClassifier - no hyperparameter tuning, calibration

In [27]:
# xgmodel0 = XGBClassifier( n_estimators=178 )
# xgmodel0.fit(x_train, y_train)

# y_pred0 = xgmodel0.predict(x_test)
# y_pred_proba0 = xgmodel0.predict_proba(x_test)[:, 1]

# auc_0 = roc_auc_score(ytest, y_pred_proba0)  
# f1_0 = f1_score(ytest, y_pred0)
# recall0 = recall_score(ytest, y_pred0)
# conf_matrix0 = confusion_matrix(ytest, y_pred0)
# prec0 = precision_score(ytest, y_pred0)
# accuracy0 = accuracy_score(ytest,y_pred0)

# print(f" accuracy: {accuracy0},\n auc: {auc_0},\n f1: {f1_0},\n recall: {recall0},\n conf_matrix: [{conf_matrix0[0][0]}][{conf_matrix0[0][1]}]\n \t \t [{conf_matrix0[1][0]}][{conf_matrix0[1][1]}] ,\n precision: {prec0}")

 accuracy: 0.972282509017546,
 auc: 0.9916833682106606,
 f1: 0.8700372653002724,
 recall: 0.9839127968230813,
 conf_matrix: [230177][6857]
 	 	 [397][24281] ,
 precision: 0.7797867557325454


## XGB Model with hyper-parameteres

In [39]:
def hyperparameter_tuning(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 10, 30)
    learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
    subsample = trial.suggest_float('subsample', 0.4, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.4, 1.0)
    gamma = trial.suggest_float('gamma', 0.0, 5.0)
    reg_lambda = trial.suggest_loguniform('reg_lambda', 1e-8, 10.0)
    reg_alpha = trial.suggest_loguniform('reg_alpha', 1e-8, 1.0)
    scale_pos_weight = trial.suggest_float('scale_pos_weight', 1.0, 10.0)

    model = XGBClassifier(
        n_estimators = n_estimators,
        max_depth = max_depth,
        learning_rate = learning_rate,
        subsample = subsample,
        colsample_bytree = colsample_bytree,
        gamma = gamma,
        reg_lambda = reg_lambda,
        reg_alpha = reg_alpha,
        scale_pos_weight = scale_pos_weight
    )

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return accuracy_score(ytest, y_pred)


# study = optuna.create_study(direction='maximize')
# study.optimize(hyperparameter_tuning, n_trials=100)

# # Best hyperparameters
# print("Best hyperparameters: ", study.best_params)

# best params -> 'n_estimators':222,"max_depth": 28,"learning_rate": 0.005138598680930574,"subsample": 0.7920153738801456, "colsample_bytree": 0.8487913943315823, 
#     'gamma': 0.3867854311036625,    'reg_lambda': 3.415670332071889e-07,     'reg_alpha': 0.0022581801625353695,     "scale_pos_weight": 1.0110340593584566

## XGboost classifier with hyperparameter

In [37]:
# xgmodel1 = XGBClassifier(
#     use_label_encoder=False,
#     eval_metric='logloss',
#     n_estimators= 222,
#     max_depth= 28,
#     learning_rate= 0.005138598680930574,
#     subsample= 0.7920153738801456, 
#     colsample_bytree= 0.8487913943315823, 
#     gamma= 0.3867854311036625,
#     reg_lambda= 3.415670332071889e-07, 
#     reg_alpha= 0.0022581801625353695, 
#     scale_pos_weight= 1.0110340593584566
# )
# xgmodel1.fit(x_train, y_train)

# y_pred1 = xgmodel1.predict(x_test)
# y_pred_proba1 = xgmodel1.predict_proba(x_test)[:, 1]

# auc1 = roc_auc_score(ytest, y_pred_proba1)  # AUC should use probabilities
# f11 = f1_score(ytest, y_pred1)
# recall1 = recall_score(ytest, y_pred1)
# conf_matrix1 = confusion_matrix(ytest, y_pred1)
# prec1 = precision_score(ytest, y_pred1)
# accuracy1 = accuracy_score(ytest,y_pred1)

# print(f" accuracy: {accuracy1},\n auc: {auc1},\n f1: {f11},\n recall: {recall1},")
# print(f"\n conf_matrix: [{conf_matrix1[0][0]}][{conf_matrix1[0][1]}]\n \t \t [{conf_matrix1[1][0]}][{conf_matrix1[1][1]}] ,\n precision: {prec1}")

 accuracy: 0.9736962768233783,
 auc: 0.9921560217979899,
 f1: 0.8759192501802452,
 recall: 0.9846016695032013,

 conf_matrix: [230530][6504]
 	 	 [380][24298] ,
 precision: 0.7888448802025843


## hyperparameter tuning

## xgb with hyperparameter tuning and calibration

In [36]:
xgmodel2 = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators= 222,
    max_depth= 28,
    learning_rate= 0.005138598680930574,
    subsample= 0.7920153738801456, 
    colsample_bytree= 0.8487913943315823, 
    gamma= 0.3867854311036625,
    reg_lambda= 3.415670332071889e-07, 
    reg_alpha= 0.0022581801625353695, 
    scale_pos_weight= 1.0110340593584566
)

xgmodel2.fit(x_train, y_train)

calibrated_mod = CalibratedClassifierCV(estimator=xgmodel2, method='isotonic')
calibrated_mod.fit(x_train, y_train)
y_calib_proba = calibrated_mod.predict_proba(x_test)[:, 1]

y_calib_pred_6 = (y_calib_proba >= 0.6).astype(int) # decision threshold = 0.6
y_calib_pred_7 = (y_calib_proba >= 0.7 ).astype(int) # decision threshold = 0.7


auc6 = roc_auc_score(ytest, y_calib_pred_6)  # AUC should use probabilities
f16 = f1_score(ytest, y_calib_pred_6)
recall6 = recall_score(ytest, y_calib_pred_6)
conf_matrix6 = confusion_matrix(ytest, y_calib_pred_6)
prec6 = precision_score(ytest, y_calib_pred_6)
brier6 = brier_score_loss(ytest,y_calib_pred_6)

auc7 = roc_auc_score(ytest, y_calib_pred_7)  # AUC should use probabilities
f17 = f1_score(ytest, y_calib_pred_7)
recall7 = recall_score(ytest, y_calib_pred_7)
conf_matrix7 = confusion_matrix(ytest, y_calib_pred_7)
prec7 = precision_score(ytest, y_calib_pred_7)
brier7 = brier_score_loss(ytest, y_calib_pred_7)

print(f"auc: {auc6},\n f1: {f16},\n recall: {recall6}")
print(f"\n conf_matrix: [{conf_matrix6[0][0]}][{conf_matrix6[0][1]}]\n \t \t [{conf_matrix6[1][0]}][{conf_matrix6[1][1]}] ,\n precision: {prec6},\n brier_score_loss: {brier6}")
print("--------------")
print(f"auc: {auc7},\n f1: {f17},\n recall: {recall7},")
print(f"\n conf_matrix: [{conf_matrix7[0][0]}][{conf_matrix7[0][1]}]\n \t \t [{conf_matrix7[1][0]}][{conf_matrix7[1][1]}] ,\n precision: {prec7},\n brier_score_loss: {brier7}")

auc: 0.9682952500670818,
 f1: 0.8787625883269349,
 recall: 0.9600048626306832

 conf_matrix: [231484][5550]
 	 	 [987][23691] ,
 precision: 0.810198009643993,
 brier_score_loss: 0.02497783823439506
--------------
auc: 0.9602350177267007,
 f1: 0.8779658455493426,
 recall: 0.9416484318016046,

 conf_matrix: [232014][5020]
 	 	 [1440][23238] ,
 precision: 0.8223511925826315,
 brier_score_loss: 0.024683621691019137


# data validation method here

In [ ]:
def validation(X_validation_data_path, y_validation_path):
    pass
    # NOTE : USE 'calibrated_mod' model of predictions
    # follow steps below in sequence

    # 1st call this-> x_validation = df_x_cleaner(X_validation_data_path, ['Column3', "Column5", "Column6", "Column8", "Column15"]).drop("ID",axis=1)
    # 2nd -> y_validation_data = pd.read_csv(y_validation_path, header=0).drop('ID',axis=1)

    # x_validation[['Column0','Column2','Column16','Column18']]= x_validation[['Column0','Column2','Column16','Column18']].astype(int)
    # x_validation_scaled_updated = process_skew_outlier(x_validation, mode='train', version='initial')

    # y_calib_proba = calibrated_mod.predict_proba(x_test)[:, 1]
    # y_calib_pred = (y_calib_proba >= 0.6).astype(int) # decision threshold = 0.6

    # auc = roc_auc_score( y_validation_data, y_calib_pred)  # AUC should use probabilities
    # f1 = f1_score ( y_validation_data, y_calib_pred)
    # recall = recall_score ( y_validation_data, y_calib_pred)
    # conf_matrix = confusion_matrix ( y_validation_data, y_calib_pred)
    # prec = precision_score ( y_validation_data, y_calib_pred)
    # brier = brier_score_loss ( y_validation_data,y_calib_pred)
    
    # print(f"auc: {auc6},\n f1: {f16},\n recall: {recall6}")
    # print(f"\n conf_matrix: [{conf_matrix6[0][0]}][{conf_matrix6[0][1]}]\n \t \t [{conf_matrix6[1][0]}][{conf_matrix6[1][1]}]")
    # print(f"precision: {prec6},\n brier_score_loss: {brier6}")
validation()